In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### User id

In [2]:
sample_submission = pd.read_csv('data/test-recsys/sample_submission.csv')
sample_submission.head(3)

,Id,Predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...


In [3]:
test_user_ids = sample_submission['Id']
UNQ_test_user_ids = set(test_user_ids)
len(test_user_ids)

107068

In [4]:
len(UNQ_test_user_ids)

107068

### Топ продукты юзеров на всех данных

In [5]:
import tqdm
import fnmatch
import os

In [7]:
# del products

In [6]:
sections = ['2_1/', '2_2/', '2_3/', '2_4/', '2_5/', '2_6/']

products = None

for section in tqdm.tqdm(sections):
    print(section)
    
    tab = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_' + section), 'tab_2*.csv')

    for file_name in tqdm.tqdm(tab):

        if products is None:
            products = pd.read_csv('data/test-recsys/sbermarket_tab_' + section + file_name)[['user_id', 'product_id']]
            products = products[products['user_id'].isin(UNQ_test_user_ids)]
        else:
            frame = pd.read_csv('data/test-recsys/sbermarket_tab_' + section + file_name)[['user_id', 'product_id']]
            frame = frame[frame['user_id'].isin(UNQ_test_user_ids)]
            products = products.append(frame, ignore_index=True)

len(products)

  0%|          | 0/10 [00:00<?, ?it/s]

2_1/



  0%|          | 0/3 [00:00<?, ?it/s]

2_2/



  0%|          | 0/3 [00:00<?, ?it/s]

...

2_5/



  0%|          | 0/4 [00:00<?, ?it/s]

2_6/



100%|██████████| 6/6 [01:59<00:00, 19.97s/it]


21677070

In [8]:
sample_submission.columns = ['user_id', 'predicted']
sample_submission.head(3)

,user_id,predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...


In [9]:
test_user_product = pd.merge(products, sample_submission, on='user_id')

In [10]:
len(test_user_product)

21677070

In [11]:
groupby_0 = pd.DataFrame({'count' : test_user_product.groupby(['user_id', 'product_id']).size()}).reset_index()
groupby_0 = groupby_0.sort_values(['user_id', 'count'], ascending=False)
groupby_0

,user_id,product_id,count
15124310,3142875,9741,3
15124311,3142875,63760,1
15124312,3142875,63762,1
15124313,3142875,63763,1
15124314,3142875,67694,1
...,...,...,...
495,51,7386516,1
496,51,7724781,1
498,51,10527105,1
499,51,10527113,1


In [12]:
grouped_df = groupby_0.groupby('user_id')
grouped_lists = grouped_df['product_id'].agg(lambda column: " ".join([str(elem) for elem in column]))
print(grouped_lists)

user_id
51         14863 19562 63057 3562687 709 9979 456 661 550...
65         54728 18450 3817542 709 18439 26062 105764 771...
187                                              88084 88086
400        0 813 94333 3817507 177 407 26751 55034 57085 ...
576        177 2600 102 4443 5628 10002 10757 14007 46469...
                                 ...                        
3142760    225 452 1985 5154 5601 6567 6757 8333 12817 13...
3142766    104 166 172 180 6341 14770 18438 19932 37061 5...
3142774    14616 21868 201892 1159915 1159916 4033462 504...
3142820    0 13120 13181 14105 15375 22194 38251 38931 57...
3142875    9741 63760 63762 63763 67694 68465 5044950 504...
Name: product_id, Length: 107068, dtype: object


In [13]:
len(grouped_lists)

107068

In [14]:
test_submit = pd.merge(sample_submission, grouped_lists, on='user_id', how='left')
test_submit = test_submit.drop(columns=['predicted'])
test_submit.head(3)

,user_id,product_id
0,51,14863 19562 63057 3562687 709 9979 456 661 550...
1,65,54728 18450 3817542 709 18439 26062 105764 771...
2,766,3043457 3041849 0 3042629 4033676 3497419 981 ...


In [15]:
test_submit.rename(columns={'user_id': 'Id', 'product_id': 'Predicted'})\
    .to_csv('test_submission_04.csv', sep=',', header=True, index=False)

### Выбрать топ продукты за все время

In [64]:
sections = ['2_1/', '2_2/', '2_3/', '2_4/', '2_5/', '2_6/']

products_all = None

for section in tqdm.tqdm(sections):
    print(section)
    
    tab = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_' + section), 'tab_2*.csv')

    for file_name in tqdm.tqdm(tab):

        if products_all is None:
            products_all = pd.read_csv(
                'data/test-recsys/sbermarket_tab_' + section + file_name)[['user_id', 'product_id', 'product_name']]
            products_all = products_all[products_all['user_id'].isin(UNQ_test_user_ids)][['product_id', 'product_name']]
        else:
            frame = pd.read_csv(
                'data/test-recsys/sbermarket_tab_' + section + file_name)[['user_id', 'product_id', 'product_name']]
            frame = frame[frame['user_id'].isin(UNQ_test_user_ids)][['product_id', 'product_name']]
            products_all = products_all.append(frame, ignore_index=True)

len(products_all)

  0%|          | 0/10 [00:00<?, ?it/s]

2_1/



  0%|          | 0/3 [00:00<?, ?it/s]

2_2/



  0%|          | 0/3 [00:00<?, ?it/s]

...

2_5/



  0%|          | 0/4 [00:00<?, ?it/s]

2_6/



100%|██████████| 6/6 [01:58<00:00, 19.69s/it]


21677070

In [66]:
products_all.head(3)

,product_id,product_name
0,3069043,Вода минеральная Славяновская среднегазированн...
1,3041849,"Молоко Маруся пастеризованное 3,5-4,5% 930 мл"
2,216,Творог Благода Крестьянский 12% 180 г


In [67]:
groupby_0 = pd.DataFrame({'count' : products_all.groupby(['product_id', 'product_name']).size()}).reset_index()
groupby_0 = groupby_0.sort_values(['count'], ascending=False)
groupby_0

,product_id,product_name,count
193,709,Бананы,139570
9290,39590,Сметана Простоквашино 15 % 315 г бзмж,44804
71,166,Молоко Простоквашино Отборное пастеризованное ...,36970
10588,55133,Помидоры,34424
361,1300,Сахар Aro белый кристаллический 1 кг,32142
...,...,...,...
80625,7323563,Тесто Billa дрожжевое 600 г,1
80626,7323572,Пастила Billa ванильно-сливочная 220 г,1
87328,7470328,Подкладка Limpopo для лепки настольная в ассор...,1
87327,7470327,Подкладка Limpopo для лепки в ассортименте А5,1


In [68]:
top_products = groupby_0['product_id'][:50].tolist()
top_products = [str(product) for product in top_products]
# top_products

In [69]:
def fill_with_top_products(x):
    ids = x.split()
    if len(ids) < 50:
        ids.extend(top_products[:50 - len(ids)])
        
    return " ".join(ids)

In [70]:
len(fill_with_top_products("12 34 5").split())

50

In [71]:
test_submit['product_id'] = test_submit['product_id'].apply(lambda x: fill_with_top_products(x))

In [72]:
test_submit.head()

,user_id,product_id
0,51,14863 19562 63057 3562687 709 9979 456 661 550...
1,65,54728 18450 3817542 709 18439 26062 105764 771...
2,766,3043457 3041849 0 3042629 4033676 3497419 981 ...
3,1132,95465 2629 9970 91008 2599278 1199 1583 2165 2...
4,1578,22161 19924 304 4513 45457 4689 19562 69131 25...


In [73]:
test_submit['length'] = test_submit['product_id'].apply(lambda x: len(x.split()))

In [74]:
test_submit.head()

,user_id,product_id,length
0,51,14863 19562 63057 3562687 709 9979 456 661 550...,501
1,65,54728 18450 3817542 709 18439 26062 105764 771...,176
2,766,3043457 3041849 0 3042629 4033676 3497419 981 ...,305
3,1132,95465 2629 9970 91008 2599278 1199 1583 2165 2...,103
4,1578,22161 19924 304 4513 45457 4689 19562 69131 25...,683


In [75]:
(test_submit['length'] >= 50).all()

True

In [76]:
test_submit = test_submit.drop(columns=['length'])

In [77]:
test_submit.rename(columns={'user_id': 'Id', 'product_id': 'Predicted'})\
    .to_csv('test_submission_07.csv', sep=',', header=True, index=False)

### Users

In [ ]:
# toko users

In [120]:
tab_2_1 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_1/'), 'tab_2*.csv')
print(tab_2_1)

users = set()

for file_name in tqdm.tqdm(tab_2_1):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_1/' + file_name)
    users |= set(products['user_id'])
    
len(users)

  0%|          | 0/10 [00:00<?, ?it/s]

['tab_2_products_2020-01-01.csv', 'tab_2_products_2020-01-11.csv', 'tab_2_products_2020-01-21.csv', 'tab_2_products_2020-01-31.csv', 'tab_2_products_2020-02-10.csv', 'tab_2_products_2020-02-20.csv', 'tab_2_products_2020-03-01.csv', 'tab_2_products_2020-03-11.csv', 'tab_2_products_2020-03-21.csv', 'tab_2_products_2020-03-31.csv']


100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


144261

In [121]:
tab_2_2 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_2/'), 'tab_2*.csv')
# print(tab_2_2)

# users = set()

for file_name in tqdm.tqdm(tab_2_2):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_2/' + file_name)
    users |= set(products['user_id'])
    
len(users)

100%|██████████| 3/3 [00:16<00:00,  5.56s/it]


245368

In [122]:
tab_2_3 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_3/'), 'tab_2*.csv')
# print(tab_2_3)

# users = set()

for file_name in tqdm.tqdm(tab_2_3):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_3/' + file_name)
    users |= set(products['user_id'])
    
len(users)

100%|██████████| 3/3 [00:21<00:00,  7.22s/it]


363206

In [123]:
tab_2_4 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_4/'), 'tab_2*.csv')
# print(tab_2_4)

# users = set()

for file_name in tqdm.tqdm(tab_2_4):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_4/' + file_name)
    users |= set(products['user_id'])
    
len(users)

100%|██████████| 3/3 [00:16<00:00,  5.54s/it]


438081

In [124]:
tab_2_5 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_5/'), 'tab_2*.csv')
# print(tab_2_5)

# users = set()

for file_name in tqdm.tqdm(tab_2_5):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_5/' + file_name)
    users |= set(products['user_id'])
    
len(users)

100%|██████████| 3/3 [00:15<00:00,  5.30s/it]


521582

In [125]:
tab_2_6 = fnmatch.filter(os.listdir('data/test-recsys/sbermarket_tab_2_6/'), 'tab_2*.csv')
# print(tab_2_6)

# users = set()

for file_name in tqdm.tqdm(tab_2_6):
#     key = file_name.split('_')[-1].split('.')[0]
    
    products = pd.read_csv('data/test-recsys/sbermarket_tab_2_6/' + file_name)
    users |= set(products['user_id'])
    
len(users)

100%|██████████| 4/4 [00:22<00:00,  5.51s/it]


657502

In [126]:
len(users)

657502

In [127]:
len(users.intersection(UNQ_users))

91001

In [128]:
len(users.intersection(UNQ_test_user_ids))

107068

In [129]:
len(UNQ_test_user_ids)

107068